## Multiple linear regression

### Import the relevant libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
# import sklearn now instead of statsmodel
from sklearn.linear_model import LinearRegression

### Load the data

In [2]:
data = pd.read_csv('Data/mlr_sklearn_data.csv')
data.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [3]:
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


## Create the multiple linear regression

### Declare the dependent and independent variable

In [5]:
y = data['GPA']
x = data[['SAT', 'Rand 1,2,3']]

## Regression itself

In [6]:
reg = LinearRegression()
##Here we dont need to reshape x because it is 2d already.
reg.fit(x,y)

LinearRegression()

In [7]:
reg.coef_
#The output array contains the coefficients for SAT and Rand 1,2,3 respectively

array([ 0.00165354, -0.00826982])

In [8]:
reg.intercept_

0.29603261264909486

### Calculating the R-squared

In [9]:
reg.score(x,y)

0.40668119528142843

### To get the adjusted R-squared
#### Sklearn doesnt provide a method to calculate this by default hence we either google for a package to calculate it or defined our own formula which what we will do now.
$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$
##### n = number of observations(i.e 84), p = number of predictions/features (i.e 2)
#### R^2 = the R-squared we got from our calculation

In [10]:
x.shape

(84, 2)

In [11]:
# Define R^2
r2 = reg.score(x,y)
# Define n and p
n = x.shape[0] # i.e 84
p = x.shape[1] # i.e 2

#Now calculate adjusted r2
adjusted_r2 = 1 - (1-r2) * (n-1)/(n-p-1)
adjusted_r2

0.39203134825134023

### The results gotten here are all thesame values gotten with mathstatsmodel meaning we have worked correctly althrough.
### Given that the adjusted_r2 < r2 the conclusion is that one or more of the predictors(x) have little or no explanatory power

#### With the discovery above how do we know the particular predictor/variable that is unfit for this model? i.e that holds little or no explanatory power?

## Detecting and removing the variable that is unfit for our model

#### We use features selection for this purpose. It is a process in ML that simplifies our model for easier interpretation. We use the p-values for the variable. Any variable with a p-value > 0.05 can be disregarded as it means it holds no explanatory power. 
#### There is no built in method for this purpose in sklearn hence we need to find a way around it. We will use the feature_selection.f_regression method.
#### The F-regression will calculate the linear regression for each of the predictors against the predicted variable i.e y&x1, y&x2 etc such that we now have 2 linear regressions in the case of a multiple regression with two independent variables. It will then calculate the f-statistics for each of the regressions and return their p-values since the p-value of an f-statistics is equal to the p-value of the only independent variable in the regression.



## Import the feature/method to be used from sklearn

In [12]:
from sklearn.feature_selection import f_regression

## Use the feature by passing the variables

In [13]:
f_regression(x,y)
# This will return two arrays, the first contains the f-stat. values for x1&x2
# Second contains their corresponding p-values, which is what we need!

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [15]:
#Output only the p_values
p_values = f_regression(x,y)[1]
p_values

array([7.19951844e-11, 6.76291372e-01])

In [16]:
# Note that e-11 means *10 to the power of -11 or /10 raise to power of 11

In [17]:
# Let's round them to 3dp to save us the stress of math. calculations
p_values.round(3)

array([0.   , 0.676])

#### From the p-values now we see that the second value of x which is the Rand 1,2,3 is useless. However this doesn't translate to the usefulness of the variables in a multiple regression since we obtained it through linear regression. F-statistics should therefore be used with caution as it is more suitable for linear regressions.

## Displaying the summary table

In [26]:
reg_summary = pd.DataFrame(data = x.columns.values, columns=['Features'])
reg_summary ['Coefficients'] = reg.coef_
reg_summary ['P-values'] = p_values.round(3)
reg_summary

,Features,Coefficients,P-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676
